In [296]:
!python --version

Python 3.6.9


In [297]:
!which python

/home/ubuntu/pysparkenv/bin/python


In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

# Импорт либ для парсинга урла
import re
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote

In [298]:
# Загружаем данные для тренировки модели

with open('/home/ubuntu/lab04_train_merged_labels.json') as f:
    df = pd.DataFrame(json.loads(line) for line in f)

#df = pd.read_json('/home/ubuntu/test.json' , lines = True)

In [299]:
print(df.shape)
print(list(df.columns))

(36138, 3)
['uid', 'gender_age', 'visits']


In [300]:
def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    #if netloc is not None: return str(netloc.encode('utf-8')).strip()
    if netloc is not None: return str(netloc).strip()
    return None

In [315]:
def get_urls(list_):  
    arr = []
    for l in list_:
        for key,value in l.items():
            if key == 'url':
                arr.append(url2domain(value))
                
    #print([key for l in list_ for key in l.items])
    #print(arr)
    #','.join(arr)
    return arr

df['urls'] = df['visits'].apply(get_urls)

In [316]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36138 entries, 0 to 36137
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uid         36138 non-null  object
 1   gender_age  36138 non-null  object
 2   visits      36138 non-null  object
 3   urls        36138 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


In [323]:
target = df[['gender_age']]
data = df[['urls']]

In [324]:
# Делим данные на тренировочную и тестовую выборку

x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.1)

In [325]:
x_train

,urls
9202,"[womenhealthnet.ru, yandex.ru, mxttrf.com, mxt..."
15197,"[darkside.ru, selfcomposing.org, musicstyle.ru..."
5810,"[sasisa.ru, yaplakal.com, yaplakal.com, yaplak..."
33375,"[vseprivoroty.ru, inetbox.net, m.avito.ru, m.a..."
20411,"[online-sberbank.3dn.ru, yandex.ru, textrefera..."
...,...
34808,"[gotovim-doma.ru, smachno.ua, mothercare.ru, g..."
27158,"[m.news.yandex.ru, ridus.ru, ridus.ru, m.news...."
12166,"[rudiplom.ru, eva.ru, economy-lib.com, disserc..."
26349,"[la2.mmotop.ru, la2.mmotop.ru, la2.mmotop.ru, ..."


In [326]:
# Превращаем наши урлы в числовой вектор

def dummy(doc):
    return doc

cv = CountVectorizer(tokenizer=dummy,
    preprocessor=dummy)

cv.fit_transform(df['urls'].values)

x_train = cv.transform(x_train['urls'].values)
x_test = cv.transform(x_test['urls'].vablues)

In [329]:
cv.get_feature_names()

['&referrer=${REFERER_URL}',
 '&referrer=${referer_url}',
 '&referrer=2gis.ru',
 '&referrer=alkislarlayasiyorum.com',
 '&referrer=comcast.net',
 '&referrer=ebay.com',
 '&referrer=examiner.com',
 '&referrer=go.com',
 '&referrer=http:',
 '&referrer=indeed.co.uk',
 '&referrer=mondedestars.com',
 '&referrer=realtor.com',
 '&referrer=rightmove.co.uk',
 '&referrer=xe.com',
 '(empty)',
 '-',
 '0-50.ru',
 '0-79.ru',
 '0-dome.ru',
 '0001s00.compress.to',
 '0001s01.compress.to',
 '0001s02.compress.to',
 '0002t00.compress.to',
 '0003b00.compress.to',
 '0003b03.compress.to',
 '000a.ru',
 '001-lab.at.ua',
 '00149.ucoz.com',
 '001games.com',
 '001s00.sci-hub.ir',
 '001s02.sci-hub.ir',
 '001s03.sci-hub.ir',
 '001s04.sci-hub.ir',
 '001s05.sci-hub.ir',
 '001s06.sci-hub.ir',
 '001s08.sci-hub.ir',
 '001s09.sci-hub.ir',
 '001s1.ludocori.cl',
 '002t00.sci-hub.ir',
 '002t01.sci-hub.ir',
 '003.ratiss.org',
 '003.ru',
 '003b00.sci-hub.ir',
 '003b02.sci-hub.ir',
 '003b03.sci-hub.ir',
 '003b06.sci-hub.ir',
 '00

In [327]:
# Кодируем возраст

labelencoder = LabelEncoder()
labelencoder.fit_transform(df['gender_age'].values)

y_train = labelencoder.transform(y_train['gender_age'].values)
y_test = labelencoder.transform(y_test['gender_age'].values)

In [328]:
# Обучаем модель
model = LinearRegression().fit(x_train, y_train)

In [330]:
model.intercept_
model.coef_

array([ 0.02432487, -0.21843788,  0.74052797, ...,  0.31519302,
        1.36068212, -0.07068484])

In [331]:
# Оценка на тренировочных данных
model.score(x_train, y_train)

0.7964375435702699

In [332]:
# Оценка на тестовых данных
model.score(x_test, y_test)

-8865.307649801063

In [333]:
predict1 = model.predict(x_train)

In [334]:
predict1

array([1.00874269, 5.00273496, 8.00047276, ..., 0.97569362, 8.02369722,
       2.00499845])

In [335]:
labelencoder.inverse_transform(predict1)

ValueError: y contains previously unseen labels: [-1.87884493 -1.83845825 -1.74683694 ... 10.46854327 10.53848024
 11.28190505]

In [292]:
predict2 = model.predict(x_test)

In [293]:
predict2

array([ 6.99982440e+00,  6.99998154e+00,  1.00002953e+00,  5.99998815e+00,
        9.99986926e-01,  9.99997753e-01,  1.47460937e-06,  7.99998862e+00,
        5.99984306e+00,  6.00001844e+00,  3.99991819e+00,  5.99998121e+00,
        6.99993923e+00,  8.99999330e+00,  2.99998509e+00,  2.00001588e+00,
        9.99973280e-01,  6.00000982e+00,  6.00000006e+00,  9.99864244e-01,
        5.99997347e+00,  1.99996756e+00,  7.00000099e+00,  9.99984908e-01,
        1.00000520e+00,  3.00008833e+00,  8.00000946e+00,  1.00000310e+00,
        5.99998139e+00,  6.00000137e+00,  6.00000223e+00,  4.00032612e+00,
        5.00000009e+00,  2.99956868e+00,  5.99999792e+00,  1.99998880e+00,
        1.00011795e+00,  1.00015763e+00,  5.99996602e+00,  8.00000129e+00,
        5.99968528e+00,  1.00000328e+00,  3.00026424e+00,  5.99982551e+00,
        2.00001372e+00,  6.99956681e+00,  6.00000193e+00,  9.99952979e-01,
        6.00021337e+00,  8.99997160e+00,  5.00001643e+00,  3.99967097e+00,
       -1.64655910e-05,  

In [294]:
labelencoder.inverse_transform(predict2)

ValueError: y contains previously unseen labels: [-8.19992255e-05 -1.64655910e-05 -1.24065489e-05  1.47460937e-06
  9.99841047e-01  9.99864244e-01  9.99932342e-01  9.99952979e-01
  9.99973280e-01  9.99984908e-01  9.99986926e-01  9.99997753e-01
  1.00000310e+00  1.00000328e+00  1.00000520e+00  1.00000715e+00
  1.00001310e+00  1.00002953e+00  1.00011795e+00  1.00015763e+00
  1.00042090e+00  1.99996756e+00  1.99998880e+00  2.00001372e+00
  2.00001588e+00  2.99956868e+00  2.99984751e+00  2.99997709e+00
  2.99998509e+00  3.00008833e+00  3.00026424e+00  3.99967097e+00
  3.99991819e+00  4.00032612e+00  5.00000009e+00  5.00001643e+00
  5.99968528e+00  5.99982551e+00  5.99984306e+00  5.99986564e+00
  5.99996602e+00  5.99997347e+00  5.99998121e+00  5.99998139e+00
  5.99998815e+00  5.99999754e+00  5.99999792e+00  5.99999865e+00
  5.99999895e+00  6.00000006e+00  6.00000137e+00  6.00000193e+00
  6.00000223e+00  6.00000417e+00  6.00000811e+00  6.00000982e+00
  6.00001844e+00  6.00021337e+00  6.00023